In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import torch.optim as optim
import pandas as pd
import argparse
import json
import csv
import requests
from datetime import datetime, timedelta
import stats_getter
import importlib
import os
from zoneinfo import ZoneInfo
import platform

importlib.reload(stats_getter)
from stats_getter import getPointMargin, getPlayersInGame, getGameRotation, get_injured_players


def load_games2(api_key,
                csv_filename='draft_kings_nba_odds.csv',
                ):
    """
    Fetches historical NBA spreads into `csv_filename`, columns:
      date, game_id, home_team, away_team,
      home_money_line, home_spread, away_money_line, away_spread,
      api_call_date, season

    If the first snapshot is >24 h before tip-off, retries at tip-off–1 h.
    If still no odds, prints a warning and skips the game.
    """
        
    # --- 1) Season bounds ---
    start_date = datetime.fromisoformat("2024-10-22T15:00:00+00:00")
    end_date   = datetime.fromisoformat("2025-04-13T15:00:00+00:00")
    season     = f"{start_date.year}-{(start_date.year+1)%100:02d}"

    # --- 2) Prepare CSV ---
    headers = [
        'date','game_id','home_team','away_team',
        'home_money_line','home_spread',
        'away_money_line','away_spread',
        'api_call_date','season'
    ]
    if os.path.exists(csv_filename):
        os.remove(csv_filename)
    os.makedirs(os.path.dirname(csv_filename) or '.', exist_ok=True)

    with open(csv_filename, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(headers)

        # ——— track which games we've written so far ———
        logged_game_ids = set()
        current = start_date
        
        while current <= end_date:
            date_str = current.strftime("%Y-%m-%dT%H:%M:%SZ")
            print(f"Fetching odds for {date_str}…")
            resp = requests.get(
                'https://api.the-odds-api.com/v4/historical/sports/basketball_nba/odds',
                params={
                    'api_key':    api_key,
                    'regions':    'us',
                    'markets':    'spreads',
                    'oddsFormat': 'american',
                    'dateFormat':'iso',
                    'date':       date_str,
                }
            )
            current += timedelta(days=1)

            if resp.status_code != 200:
                print(f"  → HTTP {resp.status_code}, skipping")
                continue

            payload = resp.json()
            games = payload.get('data', []) if isinstance(payload, dict) else payload

            for game in games:
                if not isinstance(game, dict):
                    continue

                # parse tip-off UTC → PST
                try:
                    utc_commence = datetime.strptime(
                        game['commence_time'], "%Y-%m-%dT%H:%M:%SZ"
                    ).replace(tzinfo=ZoneInfo("UTC"))
                except Exception as e:
                    print(f"  → parse error for game: {e}")
                    continue
                pst_commence = utc_commence.astimezone(ZoneInfo("America/Los_Angeles"))

                # original API call timestamp → PST
                utc_api  = datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%SZ")\
                                    .replace(tzinfo=ZoneInfo("UTC"))
                pst_api  = utc_api.astimezone(ZoneInfo("America/Los_Angeles"))

                # 1) skip if after tip-off
                if pst_api > pst_commence:
                    continue

                # 2) if >24 h early skip 
                if (pst_commence - pst_api) > timedelta(hours=24):
                    continue

                # ——— 3) skip duplicates ———
                game_id = game['id']
                if game_id in logged_game_ids:
                    # already wrote this one on an earlier day
                    continue

                # ——— 4) skip if any bookmaker updated *after* tip-off ———
                too_late = False
                for bk in game.get('bookmakers', []):
                    lu = bk.get('last_update')
                    if not lu:
                        continue
                    try:
                        utc_lu = datetime.strptime(lu, "%Y-%m-%dT%H:%M:%SZ")\
                                            .replace(tzinfo=ZoneInfo("UTC"))
                    except Exception:
                        continue
                    pst_lu = utc_lu.astimezone(ZoneInfo("America/Los_Angeles"))
                    if pst_lu > pst_commence:
                        too_late = True
                        break
                if too_late:
                    # print(f"  → bookmaker data updated after tip-off, skipping {game_id}")
                    continue

                # --- 5) Extract spreads ---
                home, away = game.get('home_team'), game.get('away_team')
                home_ml = home_sp = away_ml = away_sp = None
                found = False

                for bk in game.get('bookmakers', []):
                    for m in bk.get('markets', []):
                        if m.get('key') != 'spreads':
                            continue
                        for o in m.get('outcomes', []):
                            if o.get('name') == home:
                                home_ml, home_sp = o.get('price'), o.get('point')
                            elif o.get('name') == away:
                                away_ml, away_sp = o.get('price'), o.get('point')
                        found = True
                        break
                    if found:
                        break

                if None in (home_ml, home_sp, away_ml, away_sp):
                    print(f"  → no spreads for {game_id}, skipping")
                    continue

                # --- 6) Write the row and mark as logged ---
                writer.writerow([
                    pst_commence.strftime("%Y-%m-%d"),
                    game_id,
                    home,
                    away,
                    home_ml,
                    home_sp,
                    away_ml,
                    away_sp,
                    pst_api.strftime("%Y-%m-%dT%H:%M:%S %Z"),
                    season
                ])
                logged_game_ids.add(game_id)

    # --- 7) Sort by date and rewrite ---
    df = pd.read_csv(csv_filename)
    df['_s'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
    df = df.sort_values('_s').drop(columns=['_s'])
    df.to_csv(csv_filename, index=False)
    print(f"✅ Finished writing {csv_filename}")


def getCSV(csv_filename='draft_kings_nba_odds.csv'):
    """
    Loads the CSV file and returns a Pandas DataFrame sorted by the 'date' column.
    If the file doesn't exist, an error message is printed and None is returned.
    
    Parameters:
        csv_filename (str): The path to the CSV file.
    
    Returns:
        pd.DataFrame or None: The DataFrame containing the CSV data sorted by date,
                              or None if the CSV file does not exist.
    """
    if not os.path.exists(csv_filename):
        print(f"CSV file {csv_filename} does not exist.")
        return None

    try:
        df = pd.read_csv(csv_filename)
    except Exception as e:
        print(f"Error loading CSV file: {e}")
        return None

    if 'date' in df.columns:
        try:
            # Convert the 'date' column to datetime objects
            df['date'] = pd.to_datetime(df['date'])
            # Sort the DataFrame by the 'date' column
            df.sort_values(by='date', inplace=True)
        except Exception as e:
            print(f"Error processing the 'date' column: {e}")
    else:
        print("The 'date' column was not found in the CSV.")

    return df


def dateToISO(date):
    """
    Returns the ISO 8601 format of a given date.

    params:
    date (str): The desired date (formatted like "AUG 17, 2020")
    
    Returns:
    str: The ISO 8601 formatted date ("YYYY-MM-DD")
    """
    # Convert the month part from uppercase to title case (e.g., "AUG" -> "Aug")
    formatted_date = date.title()
    # Parse the date string using the format for abbreviated month, day, and full year.
    dt = datetime.strptime(formatted_date, "%b %d, %Y")
    # Return the date in ISO 8601 format
    return dt.date().isoformat()

def convert_date(input_date_str):
    # Ensure the month is in title-case (e.g. "AUG" -> "Aug")
    formatted_input = input_date_str.title()
    
    # Parse the input date string.
    # %b for abbreviated month name, %d for day, and %Y for four-digit year.
    dt = datetime.strptime(formatted_input, "%b %d, %Y")
    
    # Choose the proper strftime format depending on your OS
    # On UNIX/Linux/Mac, use %-m and %-d to avoid leading zeros.
    # On Windows, you may need to use %#m and %#d.
    if platform.system() == 'Windows':
        output_format = "%#m/%#d/%y"  # e.g., "8/17/20"
    else:
        output_format = "%-m/%-d/%y"  # e.g., "8/17/20"
    
    return dt.strftime(output_format)
    
def getOutcome(home_team, season, date):
    """
    Returns the outcome of a bet for a specefic game from the perspective of the home_team

    params:
    home_team: string of home team's name
    season: string of desired nba season (formatted like 2019-20)
    date (str): The desired date (formatted like "AUG 17, 2020")
    """
    new_date = convert_date(date)
    
    df = pd.read_csv('draft_kings_nba_odds.csv')
    margin = getPointMargin(home_team, season, date) # home_team - away_team
    margin_as_float = float(margin)

    
    matching_rows = df[df['date'] == new_date]
    for home_team_name in matching_rows['home_team']:
        if home_team_name == home_team:
            game_info = matching_rows[matching_rows['home_team'] == home_team_name]
            home_spread = game_info['home_spread'].iloc[0]
            return (margin_as_float+home_spread) > 0


def clear_csv_with_header(csv_filename='draft_kings_nba_odds.csv'):
    """
    Clears all contents of the CSV file and writes only the header row.
    
    Parameters:
        csv_filename (str): The name/path of the CSV file to clear.
    """
    headers = ['date', 'game_id', 'home_team', 'away_team', 'home_odds', 'away_odds', 'api_call_date']
    with open(csv_filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(headers)
    print(f"{csv_filename} has been reset with headers.")

def clearNA(df: pd.DataFrame) -> pd.DataFrame:
        """
        Removes rows in the DataFrame where every entry is NaN.
        
        Parameters:
            df (pd.DataFrame): The DataFrame to clean.
            
        Returns:
            pd.DataFrame: The DataFrame after removing rows with all NaN values.
        """
        return df.dropna(how='all')

def append_season_odds(api_key: str,
                       master_csv: str = 'draft_kings_nba_odds.csv',
                       season_csv: str = 'draft_kings_nba_odds_2021_22.csv'):

    if os.path.exists(season_csv):
        os.remove(season_csv)
    load_games2(api_key=api_key, csv_filename=season_csv)

    # 2) append + dedupe into master
    master_df = pd.read_csv(master_csv)
    season_df = pd.read_csv(season_csv)

    combined = pd.concat([master_df, season_df], ignore_index=True)
    combined.drop_duplicates(['game_id','date'], inplace=True)
    
    combined['_s'] = pd.to_datetime(combined['date'], infer_datetime_format=True)
    
    combined.sort_values('_s', inplace=True)
    combined.drop('_s', axis=1, inplace=True)
    
    combined.to_csv(master_csv, index=False)
    print(f"✅ Appended {season_csv} into {master_csv}")
    

def clear_2021_2022_season(master_csv: str = 'draft_kings_nba_odds.csv') -> None:
    """
    Deletes all rows corresponding to the 2021–22 season from master_csv,
    backing them up to 'draft_kings_nba_odds_2021_22_backup.csv'.
    """
    # 1) Load the master CSV
    df = pd.read_csv(master_csv)
    original_count = len(df)

    # 2) Identify 2021–22 rows
    if 'season' in df.columns:
        mask_21_22 = df['season'] == '2021-22'
    else:
        # fallback: parse 'date' column of form M/D/YY
        dates = pd.to_datetime(df['date'], format='%m/%d/%y')
        start = datetime(2021, 10, 19)
        end   = datetime(2022, 4, 10)
        mask_21_22 = dates.between(start, end)

    # 3) Backup the 2021–22 rows
    backup_csv = 'draft_kings_nba_odds_2021_22_backup.csv'
    df.loc[mask_21_22].to_csv(backup_csv, index=False)
    removed_count = mask_21_22.sum()

    # 4) Drop them from the master DataFrame
    df_clean = df.loc[~mask_21_22].reset_index(drop=True)

    # 5) Overwrite the master CSV
    df_clean.to_csv(master_csv, index=False)

    # 6) Summary report
    remaining_count = len(df_clean)
    print((
        f"Cleared {removed_count} rows for season 2021–22 "
        f"(backed up to '{backup_csv}'); "
        f"{remaining_count} rows remain in '{master_csv}'."
    ))

    
# Parse the API key from command line arguments (or use the default)
parser = argparse.ArgumentParser(description='Sample V4')
parser.add_argument('--api-key', type=str, default='a940b73208ba1caa8499a7aaf92a4754')
args = parser.parse_args(args=[])
API_KEY = args.api_key

def remove_game_from_csv(
    csv_path: str,
    home_team: str,
    away_team: str,
    game_date: str,
    season: str = None
):
    """
    Removes a specific game from the CSV based on home team, away team, date, and optional season.

    Parameters:
    - csv_path (str): Path to your draft_kings_nba_odds.csv
    - home_team (str): Exact name of the home team (e.g. "Los Angeles Lakers")
    - away_team (str): Exact name of the away team (e.g. "Golden State Warriors")
    - game_date (str): Date of the game in "MM-DD-YY" format
    - season (str, optional): Season string if your CSV has a 'season' column (e.g. "2020-21")
    """
    # Load the CSV
    df = pd.read_csv(csv_path)

    # Ensure 'date' column is datetime.date
    df['date'] = pd.to_datetime(df['date']).dt.date

    # Parse the target date
    target_date = pd.to_datetime(game_date).date()

    # Build mask to keep rows that do NOT match the game you want to remove
    if season and 'season' in df.columns:
        mask_keep = ~(
            (df['home_team'] == home_team) &
            (df['away_team'] == away_team) &
            (df['date'] == target_date) &
            (df['season'] == season)
        )
    else:
        mask_keep = ~(
            (df['home_team'] == home_team) &
            (df['away_team'] == away_team) &
            (df['date'] == target_date)
        )

    # Filter and overwrite the CSV
    df[mask_keep].to_csv(csv_path, index=False)
    print(f"Removed game: {away_team} @ {home_team} on {game_date}" + (f" in season {season}" if season else ""))


def add_game_to_csv(
    csv_path: str,
    game_id: str,
    home_team: str,
    away_team: str,
    game_date: str,
    season: str,
    home_money: int,
    home_spread: float,
    away_money: int,
    away_spread: float,
    api_call_date: str,
    bookmaker: str
) -> None:
    """
    Appends one row to your draft_kings_nba_odds.csv.

    Parameters:
    - csv_path: Path to the CSV file.
    - game_id:       Unique ID from the odds API.
    - home_team:     Name of the home team.
    - away_team:     Name of the away team.
    - game_date:     Game date in "YYYY-MM-DD" format.
    - season:        Season string, e.g. "2020-21".
    - home_money:    American money-line for the home team.
    - home_spread:   Point spread for the home team.
    - away_money:    American money-line for the away team.
    - away_spread:   Point spread for the away team.
    - api_call_date: Timestamp of your API call, e.g. "2021-02-17T23:49:52Z".
    - bookmaker:     Bookmaker key, e.g. "sugarhouse".
    """

    # 1) Load WITHOUT parse_dates
    df = pd.read_csv(csv_path)

    # 2) Ensure the existing `date` column is real datetime.date
    df['date'] = pd.to_datetime(df['date'], errors='coerce').dt.date

    # 3) Build the new row, coercing our input date
    new_row = {
        'date':          pd.to_datetime(game_date, errors='coerce').date(),
        'game_id':       game_id,
        'home_team':     home_team,
        'away_team':     away_team,
        'home_money_line':    home_money,
        'home_spread':   home_spread,
        'away_money_line':    away_money,
        'away_spread':   away_spread,
        'api_call_date': api_call_date,
        'season':        season,
        'bookmaker':     bookmaker
    }

    # 4) Append, sort, and overwrite
    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
    df.sort_values(by='date', inplace=True)
    df.to_csv(csv_path, index=False)

    print(f"✅ Added {game_date} | {home_team} vs {away_team} ({bookmaker})")




    
# 1) Clear old 2021–22 rows
#clear_2021_2022_season('draft_kings_nba_odds.csv')

# 2) Download & append *all* snapshots
#append_season_odds(API_KEY)

# 3) In-place post-filter: keep only the closest-to-tipoff snapshot
#pick_best_snapshot(
    #input_csv='draft_kings_nba_odds.csv',
    #output_csv='draft_kings_nba_odds.csv'
#)

resp = requests.get(
    'https://api.the-odds-api.com/v4/sports/basketball_nba/odds',
    params={
        'api_key':     API_KEY,
        'regions':     'us',
        'markets':     'spreads',
        'oddsFormat':  'american',
    }
)

# Print your credit usage info
used      = resp.headers.get('x-requests-used')
remaining = resp.headers.get('x-requests-remaining')
last      = resp.headers.get('x-requests-last')

print(f"Credits used this period: {used}")
print(f"Credits remaining until reset: {remaining}")
print(f"Last call cost: {last}")

#clear_2021_2022_season()
#open('draft_kings_nba_odds_2021_22.csv', 'w').close()

#df = pd.read_csv('draft_kings_nba_odds_2021_22.csv')
#print(df.head())

# Remove Charlotte Hornets @ Los Angeles Lakers on January 9, 2025
remove_game_from_csv(
    csv_path="draft_kings_nba_odds.csv",
    home_team="Los Angeles Lakers",
    away_team="Charlotte Hornets",
    game_date="01-09-25",
    season="2024-25"
)

# Remove Houston Rockets @ Atlanta Hawks on January 11, 2025
remove_game_from_csv(
    csv_path="draft_kings_nba_odds.csv",
    home_team="Atlanta Hawks",
    away_team="Houston Rockets",
    game_date="01-11-25",
    season="2024-25"
)

# Remove Milwaukee Bucks @ New Orleans Pelicans on January 22, 2025
remove_game_from_csv(
    csv_path="draft_kings_nba_odds.csv",
    home_team="New Orleans Pelicans",
    away_team="Milwaukee Bucks",
    game_date="01-22-25",
    season="2024-25"
)


Credits used this period: 12209
Credits remaining until reset: 7791
Last call cost: 0
Removed game: Charlotte Hornets @ Los Angeles Lakers on 01-09-25 in season 2024-25
Removed game: Houston Rockets @ Atlanta Hawks on 01-11-25 in season 2024-25
Removed game: Milwaukee Bucks @ New Orleans Pelicans on 01-22-25 in season 2024-25
